In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [4]:
from sklearn.preprocessing import PolynomialFeatures
from sklearn.model_selection import train_test_split

df = pd.read_csv("../data/Advertising.csv", index_col=0)
X, y = df.drop("Sales", axis=1), df["Sales"]
model_poly = PolynomialFeatures(3, include_bias=False)
poly_features = model_poly.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(poly_features, y, test_size=0.33, random_state=42)

X_train.shape, X_test.shape, y_train.shape, y_test.shape

((134, 19), (66, 19), (134,), (66,))

In [6]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_X_train = scaler.fit_transform(X_train)
scaled_X_test = scaler.transform(X_test)

print(f"Scaled X_train mean{scaled_X_train.mean():.2f}, std {scaled_X_train.std():.2f}")
print(f"Scaled X_test mean{scaled_X_test.mean():.2f}, std {scaled_X_test.std():.2f}")

Scaled X_train mean-0.00, std 1.00
Scaled X_test mean-0.12, std 1.12


In [10]:
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, mean_absolute_error

def ridge_regression(X, penalty=0):
    model_ridge = Ridge(alpha=penalty)
    model_ridge.fit(scaled_X_train, y_train)
    y_pred = model_ridge.predict(X)
    return y_pred

y_pred = ridge_regression(scaled_X_test, penalty=0)

MSE = mean_squared_error(y_test, y_pred)
RMSE = np.sqrt(MSE)
MAE = mean_absolute_error(y_test, y_pred)
MSE, RMSE, MAE

(0.2650465950553601, 0.5148267621786576, 0.3748516441217865)

In [13]:
from sklearn.linear_model import Lasso
model_lasso = Lasso(alpha=0.1)
model_lasso.fit(scaled_X_train, y_train)
y_pred = model_lasso.predict(scaled_X_test)

print(np.sqrt(mean_squared_error(y_test, y_pred)), mean_absolute_error(y_test, y_pred))
model_lasso.coef_

0.7853962108799017 0.5735346450114956


array([ 1.89480144,  0.42062367,  0.        , -0.        ,  3.55216501,
        0.        ,  0.        ,  0.01110965,  0.        , -0.42677394,
       -0.        , -0.        ,  0.        ,  0.        , -0.        ,
        0.        ,  0.06706906,  0.        ,  0.        ])

In [14]:
from sklearn.linear_model import RidgeCV

model_ridgeCV = RidgeCV(alphas=[.0001, .001, .01, .1, .5, 1, 5, 10], scoring = "neg_mean_squared_error")
model_ridgeCV.fit(scaled_X_train, y_train)
print(model_ridgeCV.alpha_)

0.1


In [16]:
from sklearn.linear_model import LassoCV

model_lassoCV = LassoCV(eps=0.001, n_alphas=100, max_iter=10000, cv=5)
model_lassoCV.fit(scaled_X_train, y_train)

y_pred = model_lassoCV.predict(scaled_X_test)

model_lassoCV.alpha_, np.sqrt(mean_squared_error(y_test, y_pred)), mean_absolute_error(y_test, y_pred)

(0.004968802520343365, 0.5785146895301981, 0.4629188302693299)

In [17]:
from sklearn.linear_model import ElasticNetCV

model_elastic = ElasticNetCV(l1_ratio=[.1,.5,.7,.9,.95,.99,1], eps=0.001, n_alphas=100, max_iter=10000, cv=5)
model_elastic.fit(scaled_X_train, y_train)
y_pred = model_elastic.predict(scaled_X_test)

model_elastic.l1_ratio_, model_elastic.alpha_ ,np.sqrt(mean_squared_error(y_test, y_pred)), mean_absolute_error(y_test, y_pred)

(1.0, 0.004968802520343365, 0.5785146895301981, 0.4629188302693299)